In [1]:
from osgeo import gdal
import numpy as np

In [2]:
def output_same(data, template_file_name, output_name, data_type=None):
    """
This function is to output a raster using a template so that users don't need to define the driver of gdal.
    :param data: the raster data you want to save, it should be numpy array. It should be channel first
    :param template_file_name:
    :param output_name:
    :param data_type:
    """
    gtif = gdal.Open(template_file_name)
    ## get the first band in the file
    band = gtif.GetRasterBand(1)
    if (data_type == None):
        data_type = band.DataType
    if (len(data.shape) == 2):
        data = np.expand_dims(data, axis=0)
    ## get the rows and cols of the input file
    rows = gtif.RasterYSize
    cols = gtif.RasterXSize
    output_format = output_name.split('.')[-1].upper()
    if (output_format == 'TIF'):
        output_format = 'GTIFF'
    elif (output_format == 'RST'):
        output_format = 'rst'
    driver = gdal.GetDriverByName(output_format)
    outDs = driver.Create(output_name, cols, rows, len(data), data_type)
    for i in range(len(data)):
        outBand = outDs.GetRasterBand(i + 1)
        outBand.WriteArray(data[i])
    # georeference the image and set the projection
    outDs.SetGeoTransform(gtif.GetGeoTransform())
    outDs.SetProjection(gtif.GetProjection())
    outDs.FlushCache()
    ## need to release the driver
    del outDs
    return output_name

In [3]:
file_name = r'C:\Users\zliu1997\Desktop\final_project\ancillary data\c10S_060W_Clip.tif'
ds = gdal.Open(file_name)
array = ds.GetRasterBand(1).ReadAsArray()

In [4]:
array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [5]:
from datetime import date, timedelta
def date_transform(number):
    d0 = date(2014, 12, 31)
    delta = timedelta(number)
    d1 = d0+delta
    return d1

date_transform(1).year

2015

In [6]:
array.shape

(5646, 5671)

In [20]:
res_array = np.zeros_like(array)
for i in range(array.shape[0]):
    for j in range(array.shape[1]):
        date_1 = date_transform(int(array[i,j]%10000))
        intensity = array[i,j]/10000
        if((date_1.year>2022) and (date_1.month>4)):
            res_array[i,j]=1

In [16]:
type(int(array[0,0]))

int

In [23]:
np.max(res_array)

0

In [21]:
output_same(res_array,file_name,file_name.replace('.tif','_date_2022_5.tif'),gdal.GDT_Int16)

'C:\\Users\\zliu1997\\Desktop\\final_project\\ancillary data\\c10S_060W_Clip_date_2022_5.tif'